In [ ]:
import casadi as cs
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def create_dynamics(L=0.1) -> cs.Function:
    state = cs.SX.sym("x", 5)
    control = cs.SX.sym("u", 2)

    heading, v, steer = state[2], state[3], state[4]  # x, y, phi, v, psi
    acc, psi_dot = control[0], control[1]  # acceleration and yaw rate

    state_dot = cs.vertcat(
        v * cs.cos(heading),
        v * cs.sin(heading),
        v * cs.tan(steer) / L,
        acc,
        psi_dot,
    )

    return cs.Function("dynamics", [state, control], [state_dot])


def create_linearized_dynamics(L=0.1) -> cs.Function:
    state = cs.SX.sym("x", 5)
    control = cs.SX.sym("u", 2)

    dyn = create_dynamics(L=L)
    state_dot = dyn(state, control)

    A = cs.jacobian(state_dot, state)
    B = cs.jacobian(state_dot, control)

    return cs.Function("linearized_dynamics", [state, control], [A, B])


dyn = create_dynamics()
lin_dyn = create_linearized_dynamics()

In [ ]:
class MPC:
    def __init__(self) -> None:
        self.N = 20  # prediction horizon
        self.dt = 0.01  # time step
        self.L = 1  # wheelbase
        self.n_states = 5
        self.n_controls = 2

        self.dyn = create_dynamics(L=self.L)
        self.lin_dyn = create_linearized_dynamics(L=self.L)

        self.opti = cs.Opti()
        self._create()

    def _create(self):
        cost = 0
        constraints = []

        # Variables
        self.X = self.opti.variable(self.n_states, self.N + 1)
        self.U = self.opti.variable(self.n_controls, self.N)

        # Initial state
        self.X0 = self.opti.parameter(self.n_states)

        # Reference
        self.Xref = self.opti.parameter(self.n_states, self.N + 1)
        self.Uref = self.opti.parameter(self.n_controls, self.N)

        # Constraint on first state
        constraints.append(self.X[:, 0] == self.X0)

        # Cost function
        self.Q = self.opti.parameter(self.n_states, self.n_states)
        self.R = self.opti.parameter(self.n_controls, self.n_controls)

        # A, B = self.lin_dyn(self.X[:, 0], self.U[:, 0] * 0)

        for i in range(self.N):
            xe = self.X[:, i] - self.Xref[:, i]
            ue = self.U[:, i] - self.Uref[:, i]

            cost += xe.T @ self.Q @ xe + ue.T @ self.R @ ue

            # nonlinear dynamics
            fx = self.dyn(self.X[:, i], self.U[:, i])
            x_next_est = self.X[:, i] + fx * self.dt
            constraints.append(self.X[:, i + 1] == x_next_est)

            # A, B = self.lin_dyn(self.Xref[:, i], self.Uref[:, i])
            # dxdt = A @ xe + B @ ue
            # # fx = self.dyn(self.X[:, i], self.U[:, i])
            # x_next_est = self.X[:, i] + (dxdt) * self.dt
            # constraints.append(self.X[:, i + 1] == x_next_est)

        # Terminal cost
        xe = self.X[:, self.N] - self.Xref[:, self.N]
        cost += xe.T @ self.Q @ xe * 10

        # Maximum acceleration is 10 m/s^2
        constraints.append(self.opti.bounded(-10, self.U[0, :], 10))
        # Maximum steering angle is 60 degrees
        # max_steering = np.deg2rad(60)
        # constraints.append(self.opti.bounded(-max_steering, self.X[4, :], max_steering))

        self.opti.minimize(cost)
        self.opti.subject_to(constraints)

        opts = {
            "ipopt.print_level": 0,
        }

        self.opti.solver("ipopt", opts)

    def solve(self, x0, x_ref, u_ref, Q, R):
        self.opti.set_value(self.X0, x0)
        self.opti.set_value(self.Xref, x_ref)
        self.opti.set_value(self.Uref, u_ref)
        self.opti.set_value(self.Q, Q)
        self.opti.set_value(self.R, R)

        self.opti.solve()

        return self.opti.value(self.X), self.opti.value(self.U)

In [ ]:
state0 = np.array([0, 0, 0, 0, 0])
state_ref = np.array([10, 10, 0, 0, 0])

Q = np.diag([1, 1, 1, 1, 1])
R = np.diag([1, 1]) * 1e-3

mpc = MPC()

# x_ref = np.tile(state_ref, (mpc.N + 1, 1)).T
# X_ref is linear interpolation between state0 and state_ref
x_ref = np.zeros((5, mpc.N + 1))
for i in range(5):
    x_ref[i, :] = np.linspace(state0[i], state_ref[i], mpc.N + 1)
# now generate a reference heading based on the reference trajectory
x_ref[4, :-1] = np.arctan2(np.diff(x_ref[1, :]), np.diff(x_ref[0, :]))

u_ref = np.zeros((2, mpc.N))

states, controls = mpc.solve(state0, x_ref, u_ref, Q, R)

In [ ]:
sim_steps = 3000  # simulate for 10 seconds

history_states = []
history_target = []

current_state = state0.copy()
for stepi in range(sim_steps):
    ti = stepi * mpc.dt

    # target should move linearly in x
    # target should move sine wave in y
    state_ref[0] = 10
    state_ref[1] = 1  # * np.sin(2 * np.pi * ti)
    history_target.append(state_ref.copy())

    x_ref = np.zeros((5, mpc.N + 1))
    for i in range(5):
        x_ref[i, :] = np.linspace(current_state[i], state_ref[i], mpc.N + 1)
    # now generate a reference heading based on the reference trajectory
    x_ref[2, :-1] = np.arctan2(np.diff(x_ref[1, :]), np.diff(x_ref[0, :]))
    u_ref = np.zeros((2, mpc.N))

    states, controls = mpc.solve(current_state, x_ref, u_ref, Q, R)

    first_control = controls[:, 0]

    # apply first control input
    current_state = (
        current_state + dyn(current_state, first_control).toarray().flatten() * mpc.dt
    )
    # steering is limited
    # current_state[4] = np.clip(current_state[4], -np.deg2rad(60), np.deg2rad(60))

    history_states.append(current_state.copy())

In [ ]:
history_states = np.array(history_states)

plt.figure()
plt.scatter(history_states[:, 0], history_states[:, 1], label="MPC")
plt.scatter(
    np.array(history_target)[:, 0], np.array(history_target)[:, 1], label="Target"
)

In [ ]:
plt.scatter(
    np.array(history_target)[:, 0], np.array(history_target)[:, 1], label="Target"
)